# Part 1: Module 2: Prompt Engineering Lab: Customer Feedback Analysis

In this lab, you'll learn how to iteratively improve prompts for analyzing customer feedback using a variety of prompt engineering techniques. You'll see how each technique can help you get clearer, more actionable responses from a language model.

**Customer Feedback Example:**
> I ordered a coffee maker last week, but it still hasn’t arrived. The tracking link doesn’t work, and I can’t get through to customer service. I’m really frustrated.

## Setup


This notebook is compatible with both [MyBinder](https://mybinder.org/) and AWS SageMaker.

**Environment Notes:**
- **MyBinder:** You must provide your own AWS credentials (see below).
- **AWS SageMaker:** Credentials are pre-configured in your environment.

**Install dependencies:**
```python
!pip install -r requirements.txt --quiet
```

**Environment Notes:**
- **MyBinder:** You must provide your own AWS credentials (see below).
- **AWS SageMaker:** Credentials are pre-configured in your environment.

**AWS Credentials Setup:**
- **MyBinder:** Set your credentials as environment variables in a cell (do NOT share credentials):
```python
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # or your region
```
- **SageMaker:** No action needed unless you want to override the default region.

In [3]:
# MyBinder users: set your credentials here (do NOT share real keys)
import os
# os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # or your region

# Install required packages (run this cell first)
#!pip install -r requirements.txt --quiet
print("✅ Packages installed successfully!")

import boto3
# Set the Claude model ID (update as needed for your account)
model_id = 'anthropic.claude-3-5-sonnet-20241022-v2:0'  # Example: Claude 3.5 Sonnet


region = os.environ.get('AWS_DEFAULT_REGION', 'us-west-2')
bedrock = boto3.client('bedrock-runtime', region_name=region)
# You can find available model IDs using the earlier Bedrock listing cell.

customer_feedback = "I ordered a coffee maker last week, but it still hasn’t arrived. \
The tracking link doesn’t work, and I can’t get through to customer service. I’m really frustrated."

✅ Packages installed successfully!


In [2]:
import json

def call_bedrock_model(prompt, system_prompt=None):
    # Create the request body
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0,
        "top_k": 1
    }
    # Add the system prompt if provided
    if system_prompt is not None:
        request_body["system"] = system_prompt

    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType='application/json'
    )
    result = json.loads(response['body'].read())
    model_response = result['content'][0]['text'].strip()
    print('Model response:')
    print(model_response)
    return model_response

## 1. The "Bad" Prompt (Prompt Design Challenges)
This prompt is vague and lacks structure. Run it and observe the model's output. What problems do you notice?

In [3]:
bad_prompt = (
    f"{customer_feedback}\n\n"
    "What’s wrong here? Can you help?")
# Call your model here and print the response
response = call_bedrock_model(bad_prompt)

Model response:
I understand your frustration with the delayed delivery and communication issues. While I can't directly track your order or contact customer service for you, I can suggest some steps to help:

1. Double-check your order confirmation email for the correct tracking number
2. Try tracking through the carrier's website directly (USPS, FedEx, UPS, etc.)
3. Contact the seller through alternative means:
- Email
- Social media channels
- Their website's chat function
- Alternative customer service numbers

Would you like me to help you think through which of these options might work best in your situation?


## 2. CRISP Framework Practice
Now, let's structure the prompt using the CRISP framework (without role assignment).

**Instructions:**
- Read the customer feedback below.
- Summarize the main problem and suggest one practical next step the company should take to help this customer.
- Format your answer as:
    - **Summary:**
    - **Main Problem:**
    - **Suggested Next Step:**

In [6]:
crisp_prompt = (
    f"Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    f"Summarize the main problem and suggest practical next steps the company should take to help this customer and the escalation path to fix this issue permanently.\n"
    f"Please format your answer as:\n"
    f"Summary:<overview>\n"
    f"<bullet points>\n\n"
    f"Main Problem:<overview>\n"
    f"<Primary>\n"
    f"<Secondary>\n"
    f"<Tertiary>\n"
    f"<Other>\n\n"
    f"Suggested Next Steps:<overview>\n"
    f"<Immediate>\n"
    f"<Short-term>\n"
    f"<Medium-term>\n"
    f"<Long-term>\n"
    "For each section, provide a one sentence <overview> and details as bullet points ordered by priority"
)
#Call your model here and print the response
response = call_bedrock_model(crisp_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order.
• Order delivery delay beyond expected timeframe
• Non-functional tracking system
• Inaccessible customer service
• Multiple touchpoint failures creating compound frustration

Main Problem: Multiple system breakdowns creating a negative customer experience chain reaction.
Primary:
• Order fulfillment delay without customer communication
Secondary:
• Broken tracking link preventing order status visibility
Tertiary:
• Unresponsive/inaccessible customer service channels
Other:
• Lack of proactive issue resolution systems

Suggested Next Steps: Implement immediate customer resolution while addressing systemic issues.
Immediate:
• Contact customer directly with order status and estimated delivery date
• Provide working tracking information
• Offer compensation (discount/free shipping on next order)
• Assign dedicated customer service representative

Short-term:
• Audit tracking 

## 3. Role Assignment
Now, let's add a role to the prompt.

**Instructions:**
- Add a specific role to the prompt (e.g., customer support specialist).
- Observe how the model's response changes.

In [7]:
system_prompt = (
    f"You are an expert customer support specialist for an online retailer with 20 years of experience.\n"
    "Your job is to analyze customer feedback and suggest practical next steps to help the customer."
)
#Call your model here and print the response
response = call_bedrock_model(crisp_prompt, system_prompt=system_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order.
• Order delivery delay without clear status
• Non-functioning tracking system
• Inaccessible customer service
• Compounding frustration due to multiple touchpoint failures

Main Problem: Multiple system breakdowns creating a negative customer experience spiral.
Primary:
• Order tracking/delivery system failure
Secondary:
• Non-functional tracking link indicating potential system integration issues
Tertiary:
• Inaccessible customer service suggesting capacity or system problems
Other:
• Risk of customer churn due to multiple simultaneous service failures

Suggested Next Steps: Implement immediate customer resolution while addressing systemic issues.
Immediate:
• Directly contact customer within 1 hour with order status and estimated delivery
• Provide direct contact number/email for dedicated support agent
• Offer compensation (expedited shipping, discount, or gift card)
• M

## 4. Self-Evaluation
Now, let's have the model critique its own answer or ask clarifying questions.

**Instructions:**
- After providing your answer, review it for any uncertainties or missing information.
- If anything is unclear, list clarifying questions you would ask the customer.

In [8]:
self_eval_prompt = (
    f"{crisp_prompt}\n\n"
    "After your answer, review it for any uncertainties or missing information. If anything is unclear, list clarifying questions you would ask the customer."
)
#Call your model here and print the response
response = call_bedrock_model(self_eval_prompt, system_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order
• Order delivery status unknown
• Tracking functionality not working
• Customer service accessibility issues
• Customer expressing significant frustration

Main Problem: Multiple breakdowns in the order fulfillment and customer service system
Primary:
• Unable to track or locate order status
Secondary:
• Non-functional tracking link
Tertiary:
• Inaccessible customer service
Other:
• Potential loss of customer trust and satisfaction

Suggested Next Steps: Implement immediate resolution while addressing systemic issues
Immediate:
• Locate order in system and provide updated status to customer
• Offer immediate callback from customer service team
• Provide alternative tracking information or delivery timeline
Short-term:
• Issue compensation (shipping refund or discount)
• Ensure delivery within 24-48 hours
• Follow up with delivery confirmation
Medium-term:
• Audit tracking li

## 5. Prompt Chaining
Break the task into sequential steps, using the output of one prompt as input for the next.

**Step 1:** Extract all issues mentioned in the customer feedback.

In [9]:
import json

chaining_step1_prompt = (
    f"Extract all issues mentioned in the following customer feedback and output should be just a JSON list of strings:\n"
    f"{customer_feedback}\n\n"
)
#Call your model here and print the response
response = call_bedrock_model(chaining_step1_prompt)
issues = json.loads(response)

Model response:
[
  "delayed delivery",
  "tracking link not working",
  "customer service unreachable"
]


**Step 2:** For each issue identified, suggest a specific action the company should take.

In [11]:
import time
# Use the output from Step 1 as input here
for issue in issues:
    print(f"\n\nIssue: {issue}")
    prompt = (
        f"Suggest top 2 questions to help the company get to the root cause of this issue: \"{issue}\""
    )
    response = call_bedrock_model(prompt, system_prompt)
    time.sleep(5)



Issue: delayed delivery
Model response:
As an experienced customer support specialist, here are the 2 most critical questions to diagnose delivery delays:

1. "At which point in the shipping journey did the delay occur? (Order processing, carrier pickup, in-transit, or last-mile delivery)"
- This helps identify if it's an internal warehouse issue, carrier problem, or specific route bottleneck
- Allows for targeted solutions based on where the breakdown happened

2. "Is this delay pattern occurring with specific product categories, shipping carriers, or geographical regions?"
- Reveals if delays are systemic (affecting all orders) or specific to certain variables
- Helps identify if we need to change carriers for certain routes or adjust processing for specific products

These questions provide actionable data to implement effective solutions rather than just treating symptoms.


Issue: tracking link not working
Model response:
As an experienced customer support specialist, here are t

## 6. Chain-of-Thought Prompting
Guide the model to reason step-by-step.

In [12]:
cot_prompt = (
    "Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    "Think step-by-step:\n"
    "1. Restate the customer’s main concern in your own words.\n"
    "2. Identify all the issues the customer is experiencing.\n"
    "4. For each issue, suggest top 2 questions to help the company get to the root cause of this issue"
)
#Call your model here and print the response
call_bedrock_model(cot_prompt)

Model response:
Let me break this down step by step:

1. Restating the main concern:
The customer is frustrated because they haven't received their coffee maker purchase and are unable to get status updates or assistance.

2. Identifying all issues:
- Delayed delivery (ordered last week, hasn't arrived)
- Non-functional tracking link
- Inaccessible customer service

3. Questions for each issue:

Delayed delivery:
- When exactly was the order placed (date and time)?
- Was there a specific delivery timeframe promised at checkout?

Non-functional tracking link:
- When was the last time the tracking link worked?
- Has the customer received any shipping confirmation email with an order/tracking number?

Inaccessible customer service:
- What methods has the customer tried to contact customer service (phone, email, chat)?
- What times/days has the customer attempted to reach customer service?

These questions would help the company understand:
- If the delivery is actually delayed or still wi

"Let me break this down step by step:\n\n1. Restating the main concern:\nThe customer is frustrated because they haven't received their coffee maker purchase and are unable to get status updates or assistance.\n\n2. Identifying all issues:\n- Delayed delivery (ordered last week, hasn't arrived)\n- Non-functional tracking link\n- Inaccessible customer service\n\n3. Questions for each issue:\n\nDelayed delivery:\n- When exactly was the order placed (date and time)?\n- Was there a specific delivery timeframe promised at checkout?\n\nNon-functional tracking link:\n- When was the last time the tracking link worked?\n- Has the customer received any shipping confirmation email with an order/tracking number?\n\nInaccessible customer service:\n- What methods has the customer tried to contact customer service (phone, email, chat)?\n- What times/days has the customer attempted to reach customer service?\n\nThese questions would help the company understand:\n- If the delivery is actually delayed o

## 7. Multi-Path Reasoning (Tree of Thoughts)
Ask the model to consider multiple solution paths and select the best.

**Instructions:**
- List three different ways the company could address the customer’s main problem.
- For each approach, explain the pros and cons.
- Then, select the best approach and justify your choice.

In [10]:
tree_of_thoughts_prompt = (
    "Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    "List three different ways the company could address the customer’s main problem.\n"
    "For each approach, explain the pros and cons.\n"
    "Then, select the best approach and justify your choice."
)
#Call your model here and print the response
call_bedrock_model(tree_of_thoughts_prompt)

Model response:
Here are three approaches to address the customer's delivery and communication issues:

1. Immediate Replacement Shipment
Pros:
- Quick resolution for the customer
- Shows commitment to customer satisfaction
- Eliminates further waiting time

Cons:
- Additional cost for the company
- Original package might still arrive (duplicate shipment)
- Doesn't address underlying tracking/communication issues

2. Priority Customer Service Response
Pros:
- Provides personal attention to the issue
- Can investigate and explain the specific problem
- Maintains communication with customer

Cons:
- May take time to research the issue
- Customer remains without their product
- Requires additional customer service resources

3. Automated Status Update + Compensation
Pros:
- Immediate acknowledgment of the problem
- Customer gets clear timeline and compensation
- Scalable solution for similar issues

Cons:
- Less personal approach
- May not satisfy customers wanting immediate resolution
- 

"Here are three approaches to address the customer's delivery and communication issues:\n\n1. Immediate Replacement Shipment\nPros:\n- Quick resolution for the customer\n- Shows commitment to customer satisfaction\n- Eliminates further waiting time\n\nCons:\n- Additional cost for the company\n- Original package might still arrive (duplicate shipment)\n- Doesn't address underlying tracking/communication issues\n\n2. Priority Customer Service Response\nPros:\n- Provides personal attention to the issue\n- Can investigate and explain the specific problem\n- Maintains communication with customer\n\nCons:\n- May take time to research the issue\n- Customer remains without their product\n- Requires additional customer service resources\n\n3. Automated Status Update + Compensation\nPros:\n- Immediate acknowledgment of the problem\n- Customer gets clear timeline and compensation\n- Scalable solution for similar issues\n\nCons:\n- Less personal approach\n- May not satisfy customers wanting immedi

---

## Note: Enabling Explicit Reasoning with Claude 3.7 Sonnet on AWS Bedrock

With Claude 3.7 Sonnet on AWS Bedrock, you can enable explicit step-by-step reasoning ("extended thinking") using special API parameters.

- Set `thinking: true` and specify a `budget_tokens` value in your API request. thinking mode is OFF by default. 
- The model will return its reasoning process in separate `thinking` blocks, in addition to the final answer.
- You cannot use `temperature`, `top_p`, or `top_k` with reasoning mode enabled.

#### Best Practices for Reasoning Mode (`thinking`) in Claude 3.7 Sonnet

- **Always set** `budget_tokens < max_tokens`.
- **Minimum** `budget_tokens`: **1,024** tokens  
  _Anthropic recommends at least **4,000** tokens for more comprehensive and nuanced reasoning._
- **For multi-turn conversations:**  
  Preserve and return the **complete, unmodified `thinking` blocks** to maintain context and reasoning continuity.

#### Important Notes
- **Incompatible parameters:**: You **cannot use** `temperature`, `top_p`, or `top_k` with reasoning mode enabled.
- **Billing:**: You are **billed for thinking tokens** as output tokens. They also count towards your service quota token per minute (TPM) limit.
- **Redacted thinking blocks:**: May appear if the reasoning output does not meet safety standards.
- **Multi-turn conversations:**: Thinking blocks associated with earlier assistant messages do **not** get charged as input tokens.
- **Increased latency**: Plan for longer response times due to extra processing and output generation and adjust your client timeouts accordingly.

**Example API call:**
```json
{
  "modelId": "anthropic.claude-3-7-sonnet",
  "messages": [...],
  "thinking": true,
  "budget_tokens": 4000,
  "max_tokens": 8000
}
```

For more details, see the [AWS Bedrock Claude 3.7 Sonnet Thinking Blocks documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-37.html#model-parameters-anthropic-claude-37-thinking-blocks).


## Student Reflection
Which technique produced the most useful or reliable answer for this scenario? How did the model’s responses change as you refined your prompts?